# Instalaciones

In [6]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-qwdkhi27
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-qwdkhi27
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Carga de imágenes

In [3]:
from PIL import Image

imgs = [
    Image.open("result_Canny.png"),
    Image.open("result_Depth.png"),
    Image.open("result_Pose.png"),
]
prompts = [
    " the downtown from cyberpunk city at night with a car and person with a arm, with neon colors and with fish holograms at the center with canny condition",
    " the downtown from cyberpunk city at night with a car and person with a arm, with neon colors and with fish holograms at the center with depth condition",
    " the downtown from cyberpunk city at night with a car and person with a arm, with neon colors and with fish holograms at the center with Pose condition",
]


# Métrica CLIPScore

In [8]:
import clip
import torch
from PIL import Image

# Carga modelo
model, preprocess = clip.load("ViT-B/32", device="cuda" if torch.cuda.is_available() else "cpu")
device = next(model.parameters()).device

# Prepara imagen y texto
img = preprocess(Image.open("result_Canny.png")).unsqueeze(0).to(device)
text = clip.tokenize(["the downtown from cyberpunk city at night with a car and person with a arm, with neon colors and with fish holograms at the center with canny condition"]).to(device)

# Sin gradientes
with torch.no_grad():
    img_feat = model.encode_image(img)
    txt_feat = model.encode_text(text)
    sim = torch.cosine_similarity(img_feat, txt_feat).item()

print(f"CLIPScore: {sim:.4f}")


CLIPScore: 0.3548


In [10]:
import clip
import torch
from PIL import Image

# Carga modelo
model, preprocess = clip.load("ViT-B/32", device="cuda" if torch.cuda.is_available() else "cpu")
device = next(model.parameters()).device

# Prepara imagen y texto
img = preprocess(Image.open("result_Depth.png")).unsqueeze(0).to(device)
text = clip.tokenize(["the downtown from cyberpunk city at night with a car and person with a arm, with neon colors and with fish holograms at the center with depth condition"]).to(device)

# Sin gradientes
with torch.no_grad():
    img_feat = model.encode_image(img)
    txt_feat = model.encode_text(text)
    sim = torch.cosine_similarity(img_feat, txt_feat).item()

print(f"CLIPScore: {sim:.4f}")

CLIPScore: 0.3006


In [11]:
import clip
import torch
from PIL import Image

# Carga modelo
model, preprocess = clip.load("ViT-B/32", device="cuda" if torch.cuda.is_available() else "cpu")
device = next(model.parameters()).device

# Prepara imagen y texto
img = preprocess(Image.open("result_Pose.png")).unsqueeze(0).to(device)
text = clip.tokenize(["the downtown from cyberpunk city at night with a car and person with a arm, with neon colors and with fish holograms at the center with pose condition"]).to(device)

# Sin gradientes
with torch.no_grad():
    img_feat = model.encode_image(img)
    txt_feat = model.encode_text(text)
    sim = torch.cosine_similarity(img_feat, txt_feat).item()

print(f"CLIPScore: {sim:.4f}")

CLIPScore: 0.3599


# Métrica simétria

In [5]:
import numpy as np
from skimage.metrics import structural_similarity as ssim

for i, img in enumerate(imgs):
    arr = np.array(img.convert("L"))  # escala de grises
    midpoint = arr.shape[1] // 2
    left, right = arr[:, :midpoint], np.fliplr(arr[:, midpoint:])
    score = ssim(left, right, data_range=right.max() - right.min())
    print(f"Imagen {i+1} simetría (SSIM): {score:.4f}")


Imagen 1 simetría (SSIM): 0.1051
Imagen 2 simetría (SSIM): 0.1258
Imagen 3 simetría (SSIM): 0.0881
